<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [60]:
!pip install gekko
!pip install ipynb


**Importing libraries needed**

In [61]:
import numpy as np
import time
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**SAWMILL CLASS**

In [63]:
class Inventory:
  def __init__(self, Av, Dist, Wt=-1, Pt=-1):
    self.Available = []
    self.Available.append(Av) #Dataframe that contains Available wood Inventory at diferent time t
    self.Dist = Dist #Matrix Distance from any cell to another cell
    self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the Refinery
    self.Harvested = [] #Ammount of harvested wood
    self.Residues = []  #Residues generated by harvesting
    self.Growth = [] #Growth ammount for each hexagon (feet cubic) 
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.Candidates=[] #Candidate location for biorefinery
    self.Radio_Mill = 40
    self.Radio_Refinery = 70
   
  def Harvest_Mill_Close(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,t,self.Radio_Mill)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t, self.Radio_Mill)
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat 
    ans = a

    #Storing Harvested 
    self.Harvested[t].loc[:,('Harvested')] = 0 
    print('Reseting stage:',t+1,' Mill: ',Id1,sum(self.Harvested[t].loc[:,("Harvested")]))   

    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = ans.loc[j,('Harvested')]  
    print('Wood Harvested in stage:',t,sum(self.Harvested[t].loc[:,("Harvested")]))
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
  def Harvest_Mill_Dense(self,t, j):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')]
    a =self.Cell_Less(Id1,t,self.Radio_Mill)
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    print('Demanda:', Dem)
    self.Available_Mill = self.Disp_Radio( Id1, t,self.Radio_Mill)
    
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    prod0 = m.Intermediate(-m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat 
    ans = a

    #Storing Harvested 
    self.Harvested[t].loc[:,('Harvested')] = 0 
    print('Reseting stage:',t+1,' Mill: ',Id1,sum(self.Harvested[t].loc[:,("Harvested")]))   
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = ans.loc[j,('Harvested')]  
    print('Wood Harvested in stage:',t,sum(self.Harvested[t].loc[:,("Harvested")]))
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
     

  def Harvest_All_Mills(self,t,k):
  #Harvest k mills at year t
    self.Available.append(self.Available[t])
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill_Dense(t,i)
  
    self.Harvested.append(self.Harvested[t]) 
    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')])
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.05)
    print('Total Wood Harvested at stage: ',t, sum(self.Harvested[t].loc[:,('Harvested')]))
    print('Total Residues at stage: ',t, sum(self.Residues[t].loc[:,('Harvested')]))

  
  def Inventory_Growth(self,t,rate = 0):
    if (rate == 0):
      r = self.Growth
    else:
      r = rate
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]*(1+r/100)

  
  def Cell_Less(self, Id_Mill, t, Radio=20):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Cell_Less_Res(self, Id_Mill, t, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Disp_Radio(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  

  def Disp_Radio_Res(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    return(Disp)


  def Num_Cell(self, Id_Mill, Radio=10):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    return(len(Ans))
  
  
  def Optimal_Location(self,t,Ms,Demanda=10000000):
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization
    Num_Cand = len(self.Candidates)
    Cost=[]
    for i in range(Ms):
      Id1 =self.Candidates[i]
      a =self.Cell_Less_Res(Id1,t,self.Radio_Refinery)
      self.Dist_Refi = a.loc[:, ('DEST','DIST')]
      Num = len(self.Dist_Refi)
      H = m2.Array(m2.Var,(Num),value = 0)
      Dem = Demanda

      self.Available_Refi = self.Disp_Radio_Res( Id1, 0, self.Radio_Refinery)
      
      for i in range(Num):
        H[i].lower = 0
        H[i].upper = self.Available_Refi[i]
        
      prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
      
      m2.Equation(Dem ==m2.sum([H[i] for i in range(0,len(H))]))
      m2.Obj(prod0) 
      m2.solve(disp=False,debug=False)
      a = self.Dist_Refi
      flat = [item for l in H for item in l]
      a['Harvested'] = flat
      res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))  #Delivery cost to refinery
      Cost.append(res)
    print('Cost', Cost)
    print('Minimo en:', Cost.index(min(Cost)))

    

**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [83]:
# Main Program
start_time = time.time()
#Upload distances from each hexagon to other
Hardwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Sawlog.csv")
#Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp.csv")
#Softwood_Sawlog_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Sawlog.csv")
#Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp.csv")


Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Distance_TN.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN.csv")
print(Mills_Data.loc[:,('s_h')])


i1 = Mills_Data['plant'] == "Composite Panel"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Pulpmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Sawmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*83.3  #transforming mbf to cubic feet
i1 = Mills_Data['plant'] == "Venner"
Mills_Data.loc[i1,('Demand')] = 1600*83.3 #transforming mbf to cubic feet

YEARS = 1

#Initializing Inventories
Hardwood_Sawlog = Inventory(Hardwood_Sawlog_Data,Dist_Data,Wt='H')
Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H')
Softwood_Sawlog = Inventory(Softwood_Sawlog_Data,Dist_Data,Wt='S')
Softwood_Pulp = Inventory(Softwood_Pulp_Data,Dist_Data,Wt='S')


r1 = len(Hardwood_Sawlog.Available[0])
Id_li = list(Hardwood_Sawlog.Available[0].loc[:,("Unique_ID")])
Ha_li =[0]*r1
Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
Hardwood_Sawlog.Harvested.append(pd.DataFrame(Data))
Num_Mills = len(Mills_Data)
Num_Mills = 4

ma = Dist_Data['ORIG'].unique()
Hardwood_Sawlog.Candidates = ma
Num_Cand = len(ma)

#Assigning Candidates Id to Mills Id while getting the mills exact distance
for k in range(Num_Mills):
  Mills_Data.loc[k,('Unique_ID')] = ma[k % Num_Cand]
  
Hardwood_Sawlog.Mills=Mills_Data
for y in range(YEARS):
  Hardwood_Sawlog.Harvest_All_Mills(y,Num_Mills)    
  
#save harvested wood in file out.csv
Hardwood_Sawlog.Harvested[0].to_csv('/content/drive/My Drive/Colab Notebooks/out.csv')
print ("My program took", time.time() - start_time, "to run")

0        S
1        S
2        H
3        S
4      S/H
      ... 
259      H
260      H
261      S
262      H
263      H
Name: s_h, Length: 264, dtype: object
